In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip3 install pyspark==3.0.0

<h1><center>Инициализация</center></h1>

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, col, max, sum, desc, countDistinct
import re
from typing import List
import pyspark.sql as sql

In [ ]:
spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()

<h1><center>Загрузка данных</center></h1>

In [ ]:
import os
prog_path = '/content/programming-languages.csv'
posts_path = '/content/posts_sample.xml'

In [ ]:
posts = spark.read.format('xml').options(rowTag='row').load(posts_path)

In [ ]:
program = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option("DateTimeFormat", 'M/d/y H:m') \
      .csv(prog_path)

<h1><center>Задание</center></h1>

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: RDD API, Dataset API, SQL API.

In [ ]:
# Функция для извлечения тегов из строки
def get_tags(tags_string):
    # Проверка на пустую строку или None
    if not tags_string:
        return []
    # Регулярное выражение для извлечения тегов
    pattern = r'<([^>]+)>'
    # Извлечение тегов из строки
    tags = re.findall(pattern, tags_string)
    return tags

# Функция для извлечения года из даты и времени
def get_year(date_and_time):
    return date_and_time.year

get_tags_udf = udf(get_tags, ArrayType(StringType()))
get_year_udf = udf(get_year, IntegerType())

# Выбираем необходимые столбцы
posts_data_simplified = posts \
    .withColumn("tags", get_tags_udf(posts["_Tags"])) \
    .withColumn("year", get_year_udf(posts["_LastActivityDate"])) \
    .select("tags", "year", F.col("_ViewCount").alias("views"))

# Получаем первые 10 строк
first_rows = posts_data_simplified.limit(10).collect()

for i, row in enumerate(first_rows):
    print(i + 1, row)

1 Row(tags=['c#', 'floating-point', 'type-conversion', 'double', 'decimal'], year=2019, views=42817)
2 Row(tags=['html', 'css', 'internet-explorer-7'], year=2019, views=18214)
3 Row(tags=[], year=2017, views=None)
4 Row(tags=['c#', '.net', 'datetime'], year=2019, views=555183)
5 Row(tags=['c#', 'datetime', 'time', 'datediff', 'relative-time-span'], year=2019, views=149445)
6 Row(tags=[], year=2018, views=None)
7 Row(tags=['html', 'browser', 'timezone', 'user-agent', 'timezone-offset'], year=2019, views=176405)
8 Row(tags=['.net', 'math'], year=2018, views=123231)
9 Row(tags=[], year=2010, views=None)
10 Row(tags=[], year=2010, views=None)


In [72]:
# Разбиваем список тегов на отдельные строки и выбираем необходимые столбцы
posts_data_sorted = posts_data_simplified \
    .select("year", explode("tags").alias("tag"), "views")

# Группируем данные по году и тегам, суммируя просмотры для каждого тега в пределах одного года
posts_data_sorted = posts_data_sorted \
    .groupBy("year", "tag") \
    .agg(sum("views").alias("total_views"))

# Сортируем результаты по году и общему количеству просмотров
posts_data_sorted = posts_data_sorted \
    .orderBy("year", desc("total_views"))

posts_data_sorted.show()

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|            database|      19682|
|2008|               local|      19682|
|2008|                java|      11532|
|2008|         inheritance|      10971|
|2008|       accessibility|       7700|
|2008|           variables|       7700|
|2008|               excel|       6540|
|2008|          automation|       6540|
|2008|           interface|       3271|
|2008|      castle-windsor|       2927|
|2008|       configuration|       2927|
|2008|dependency-injection|       2927|
|2008|               linux|       2393|
|2008|                ruby|       1843|
|2008|       ruby-on-rails|       1843|
|2008|            .net-3.0|       1432|
|2008|            .net-2.0|       1432|
|2008|  visual-studio-2008|       1432|
+----+--------------------+-----------+
only showing top 20 rows



In [73]:
window = Window.partitionBy("year").orderBy(desc("total_views"))
result_df = (
    posts_data_sorted
    .withColumn("rank", rank().over(window))
    .filter("rank <= 5")
    .select("year", "tag", "total_views")
    .orderBy("year", desc("total_views"))
)

result_df.show()
result_df.write.parquet("top_10_languages_report.parquet")

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|            database|      19682|
|2008|               local|      19682|
|2008|                java|      11532|
|2009|                  c#|      73661|
|2009|                .net|      39167|
|2009|              python|      32219|
|2009|                 c++|      29381|
|2009|            winforms|      25670|
|2010|                  c#|     128597|
|2010|              arrays|      80868|
|2010|                java|      53333|
|2010|              matlab|      51865|
|2010|multidimensional-...|      51865|
|2011|                  c#|     238076|
|2011|                java|     121315|
|2011|                .net|     120734|
|2011|                 css|     119302|
|2011|             android|     107283|
+----+--------------------+-----------+
only showing top 20 rows



In [74]:
import shutil
directory_path = "top_10_languages_report.parquet"
# Удаляем директорию
shutil.rmtree(directory_path)